In [1]:
import os
import pdb
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import random
import ipdb

In [2]:
class KITTIDataset(Dataset):
    def __init__(self, root_dir, debug=False):
        """
        Args:
            root_dir (string): Directory with all the point clouds.
        """
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
            
        if debug:
            random_sample = random.randint(1, 7400)
            self.files = [self.files[random_sample]]
        

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        point_cloud_path = os.path.join(self.root_dir, self.files[idx])
        point_cloud = self.load_point_cloud_from_bin(point_cloud_path)  # Assuming point clouds are stored as .pt files
        return point_cloud
    
    def load_point_cloud_from_bin(self, root_dir):
        with open(bin_path, 'rb') as f:
            content = f.read()
            point_cloud = np.frombuffer(content, dtype=np.float32)
            point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
        return torch.from_numpy(point_cloud)
        

train_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced'
test_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/testing/velodyne_reduced'
# TODO: Add label dir

train_set = KITTIDataset(root_dir=train_dir, debug=False)
test_set = KITTIDataset(root_dir=test_dir, debug=False)



In [11]:
class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
        self.pillars_dict = {}

    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]

        # Calculate the number of pillars in x and y directions
        num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])

        
        # Using numpy's digitize to find the interval/bin each point belongs to.
        x_indices = torch.tensor(np.digitize(points[:, 0], np.linspace(self.x_min, self.x_max, num_x_pillars))) - 1
        y_indices = torch.tensor(np.digitize(points[:, 1], np.linspace(self.y_min, self.y_max, num_y_pillars))) - 1

        # TODO: Calculate pillar x-y center:
        pillar_x_center = x_indices * self.pillar_size[0] + self.pillar_size[0] / 2.0
        pillar_y_center = y_indices * self.pillar_size[1] + self.pillar_size[1] / 2.0 

        pillars = torch.zeros((num_x_pillars, num_y_pillars, self.max_points_per_pillar, self.aug_dim))
        
        # Count how many points are in each pillar to ensure we don't exceed `max_points_per_pillar`
        count = torch.zeros((num_x_pillars, num_y_pillars), dtype=torch.long)
        
        # TODO: Store points in the pillars in a vectorized way filling the pillars tensor:        
        for i in range(points.shape[0]):
            x_ind = x_indices[i]
            y_ind = y_indices[i]
            
            if count[x_ind, y_ind] < self.max_points_per_pillar:
                # Compute x_c, y_c and z_c
                x_c = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0) - points[i, 0]
                y_c = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0) - points[i, 1]
                z_c = (self.z_min + self.z_max) / 2 - points[i, 2] # assuming the z-center is the midpoint
                
                # Calculate pillar center
                x_pillar_center = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0)
                y_pillar_center = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0)
                
                # Add original x, y, and z coordinates, then x_c, y_c, z_c
                pillars[x_ind, y_ind, count[x_ind, y_ind], :3] = points[i, :3]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c]) # Sus
                pillars[x_ind, y_ind, count[x_ind, y_ind], 6] = x_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 0]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 7] = y_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 1]
                
                count[x_ind, y_ind] += 1
        
        # Zero-padding if too few point, random sampling if too many points:
        for i in range(num_x_pillars):
            for j in range(num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars 


            
# Pillarization testing:

# TODO: Get a random sample:
def load_point_cloud_from_bin(bin_path):
    with open(bin_path, 'rb') as f:
        content = f.read()
        point_cloud = np.frombuffer(content, dtype=np.float32)
        point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
    
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]
    
    return point_cloud

# Example usage
random_sample = random.randint(1, 7400)
bin_path = os.path.join(train_dir, '000039.bin')
point_cloud = load_point_cloud_from_bin(bin_path)

# TODO: Pillarize the sample:
D = 9 # Augmented dimension
N = 100 # Max number of points per pillar

pillarizer = Pillarization(aug_dim=D, x_min=-40.0, x_max=40.0, y_min=-25.0, y_max=25.0, 
           z_min=-3, z_max=3, pillar_size=(0.5, 0.5), max_points_per_pillar=N)

pillars = pillarizer.make_pillars(torch.from_numpy(point_cloud))      
    

> /tmp/ipykernel_66526/3275739938.py(82)make_pillars()
     81         ipdb.set_trace()
---> 82         return pillars
     83 

ipdb> pillars[0,1,:].size()
torch.Size([100])
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
ipdb> exit


In [10]:
pillars.size()

torch.Size([9, 63648, 100])

In [6]:
# Pillarization pipeline:

class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
    
    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]
        
        # Create pillar grid
        x_indices = ((points[:, 0] - self.x_min) // self.pillar_size[0]).type(torch.int64)
        y_indices = ((points[:, 1] - self.y_min) // self.pillar_size[1]).type(torch.int64)

        # Calculate the number of pillars in x and y directions
        num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

        # Create a tensor to hold the pillars data
        #pillars = torch.zeros((num_x_pillars, num_y_pillars, self.max_points_per_pillar, points.shape[1]))
        pillars = torch.zeros((num_x_pillars, num_y_pillars, self.max_points_per_pillar, self.aug_dim))
        indices_dict = {}
        
        # Fill the pillars:
        for i in range(points.shape[0]):
            x_ind, y_ind = x_indices[i], y_indices[i]
            if pillars[x_ind, y_ind].shape[0] < self.max_points_per_pillar:
                # Calculate mean coordinates of the points inside this pillar
                x_mean = torch.mean(points[x_indices == x_ind, 0])
                y_mean = torch.mean(points[y_indices == y_ind, 1])
                
                # Calculate pillar x-y center:
                pillar_x_center = x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0
                pillar_y_center = y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0
                
                # Augment the point with its offset from the pillar's center
                augmented_point = torch.cat((points[i, :3], points[i, 3:], 
                 points[i, :3] - torch.tensor([x_mean, y_mean, 0]),
                 points[i, :2] - torch.tensor([pillar_x_center, pillar_y_center])))
                
                # Insert the augmented point into the pillar tensor
                indices_dict[x_ind, y_ind] = i 
                pillars[x_ind, y_ind, pillars[x_ind, y_ind].shape[0]] = augmented_point
        
                
                
        # Random sampling or zero padding
        for i in range(num_x_pillars):
            for j in range(num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars
    
    #def visualization(pillars):
        
        
        

        
# Pillarization testing:

# TODO: Get a random sample:
def load_point_cloud_from_bin(bin_path):
    with open(bin_path, 'rb') as f:
        content = f.read()
        point_cloud = np.frombuffer(content, dtype=np.float32)
        point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
    
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]
    
    return point_cloud

# Example usage
random_sample = random.randint(1, 7400)
bin_path = os.path.join(train_dir, '000038.bin')
point_cloud = load_point_cloud_from_bin(bin_path)

# TODO: Pillarize the sample:
D = 9 # Augmented dimension
N = 100 # Max number of points per pillar

pillarizer = Pillarization(aug_dim=D, x_min=0, x_max=70.4, y_min=-40.8, y_max=40.8, 
           z_min=-3, z_max=3, pillar_size=(0.2, 0.2), max_points_per_pillar=N)

pillars = pillarizer.make_pillars(torch.from_numpy(point_cloud))
        

NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
NO
YES
NO
YES
NO
YES
NO
YES
NO
YES
NO
YES
NO
YES
NO
NO
NO
NO
NO
NO
YES
NO
NO
NO
NO
NO
YES
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
YES
YES
YES
NO
NO
NO
NO
YES
NO
NO
NO
NO
YES
NO
YES
YES
YES
NO
YES
NO
NO
YES
NO
YES
NO
NO
YES
NO
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
NO
YES
YES
NO
NO
YES
NO
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
NO
YES
YES
NO
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
NO
YES
YES
YES
NO
YES
NO
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
NO
YES
YES
YES
YES
YES
YES
YES
NO
YES
YES
YES
YES
YES
YES
NO
YES
YES
YES
YES
YES
YES
NO
YES
NO
NO
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
YES
NO
YES
YES
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
YES
NO
YES
YES
YES
N

In [ ]:
pillars.size()
        

In [ ]:
## SEPARATE

In [ ]:
# Pillarization pipeline:

class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
    
    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]
        
        # Create pillar grid
        x_indices = ((points[:, 0] - self.x_min) // self.pillar_size[0]).type(torch.int64)
        y_indices = ((points[:, 1] - self.y_min) // self.pillar_size[1]).type(torch.int64)

        # Calculate the number of pillars in x and y directions
        num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

        # Create a tensor to hold the pillars data
        #pillars = torch.zeros((num_x_pillars, num_y_pillars, self.max_points_per_pillar, points.shape[1]))
        pillars = torch.zeros((num_x_pillars, num_y_pillars, self.max_points_per_pillar, self.aug_dim))
        indices_dict = {}
        
        # Fill the pillars:
        for i in range(points.shape[0]):
            x_ind, y_ind = x_indices[i], y_indices[i]
            if pillars[x_ind, y_ind].shape[0] < self.max_points_per_pillar:
                # Calculate mean coordinates of the points inside this pillar
                x_mean = torch.mean(points[x_indices == x_ind, 0])
                y_mean = torch.mean(points[y_indices == y_ind, 1])
                
                # Calculate pillar x-y center:
                pillar_x_center = x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0
                pillar_y_center = y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0
                
                # Augment the point with its offset from the pillar's center
                augmented_point = torch.cat((points[i, :3], points[i, 3:], 
                 points[i, :3] - torch.tensor([x_mean, y_mean, 0]),
                 points[i, :2] - torch.tensor([pillar_x_center, pillar_y_center])))
                
                # Insert the augmented point into the pillar tensor
                indices_dict[x_ind, y_ind] = i 
                pillars[x_ind, y_ind, pillars[x_ind, y_ind].shape[0]] = augmented_point
        
                
                
        # Random sampling or zero padding
        for i in range(num_x_pillars):
            for j in range(num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars
    
    #def visualization(pillars):
        
        
        

        
# Pillarization testing:

# TODO: Get a random sample:
def load_point_cloud_from_bin(bin_path):
    with open(bin_path, 'rb') as f:
        content = f.read()
        point_cloud = np.frombuffer(content, dtype=np.float32)
        point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
    
    x = point_cloud[:, 0]
    y = point_cloud[:, 1]
    z = point_cloud[:, 2]
    
    return point_cloud

# Example usage
random_sample = random.randint(1, 7400)
bin_path = os.path.join(train_dir, '000038.bin')
point_cloud = load_point_cloud_from_bin(bin_path)

# TODO: Pillarize the sample:
D = 9 # Augmented dimension
N = 100 # Max number of points per pillar

pillarizer = Pillarization(aug_dim=D, x_min=0, x_max=70.4, y_min=-40.8, y_max=40.8, 
           z_min=-3, z_max=3, pillar_size=(0.2, 0.2), max_points_per_pillar=N)

pillars = pillarizer.make_pillars(torch.from_numpy(point_cloud))
        

In [ ]:
pillars.size()